In [60]:
#imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [61]:
#load the environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

#check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [62]:
#Instantiate OpenAI
openai = OpenAI()


In [63]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError

class Website:
    def __init__(self, url):
        self.url = url
        self.title = "No title found"
        self.text = ""

    @classmethod
    async def create(cls, url):
        self = cls(url)
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            
            try:
                await page.goto(url, timeout=60000)
                await page.wait_for_selector("body", timeout=30000)
                
                self.title = await page.title() or "No title found"
                body = await page.query_selector("body")
                self.text = await body.inner_text() if body else ""
            
            except TimeoutError:
                print(f"Timeout while loading: {url}")
            
            finally:
                await browser.close()
        
        return self

In [64]:
tanzir = await Website.create("https://tanzirr.netlify.app/")
print("URL:", tanzir.url)
print("Title:", tanzir.title)
print("Text:", tanzir.text[:500], "...")

URL: https://tanzirr.netlify.app/
Title: Tanzir Razzaque AI/ML Engineer Portfolio
Text: Tanzir Razzaque
About
Skills
Projects
Experience
Contact
AI/ML Engineer
Building the Future with
Artificial Intelligence

Passionate about creating intelligent systems that solve real-world problems. Specializing in machine learning, deep learning, and AI research.

View My Work
Contact
About Me
Transforming Ideas into Intelligent Solutions

I'm a passionate AI/ML engineer with a deep commitment to pushing the boundaries of artificial intelligence. With over 2 years of experience as a data analy ...


In [65]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [66]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [67]:
print(user_prompt_for(tanzir))

You are looking at a website titled Tanzir Razzaque AI/ML Engineer Portfolio
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Tanzir Razzaque
About
Skills
Projects
Experience
Contact
AI/ML Engineer
Building the Future with
Artificial Intelligence

Passionate about creating intelligent systems that solve real-world problems. Specializing in machine learning, deep learning, and AI research.

View My Work
Contact
About Me
Transforming Ideas into Intelligent Solutions

I'm a passionate AI/ML engineer with a deep commitment to pushing the boundaries of artificial intelligence. With over 2 years of experience as a data analyst and AI researcher, I specialize in building production-ready ML systems that solve real-world problems.

My expertise lies in the intersection of deep learning research and practical implementation. My work spans from developing pose estimation system

In [68]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [69]:
messages_for(tanzir)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Tanzir Razzaque AI/ML Engineer Portfolio\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nTanzir Razzaque\nAbout\nSkills\nProjects\nExperience\nContact\nAI/ML Engineer\nBuilding the Future with\nArtificial Intelligence\n\nPassionate about creating intelligent systems that solve real-world problems. Specializing in machine learning, deep learning, and AI research.\n\nView My Work\nContact\nAbout Me\nTransforming Ideas into Intelligent Solutions\n\nI'm a passionate AI/ML engineer with a deep commitment to pushing the boundaries of artificial intelligence. With over 2 years of experience as a data analyst an

In [79]:
#Summarize 
async def summarize(url):
    website = await Website.create(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [85]:
# A function to display this nicely in the Jupyter output, using markdown

async def display_summary(url):
    summary = await summarize(url)
    display(Markdown(summary))

In [86]:
await display_summary("https://tanzirr.netlify.app/")

## Summary of Tanzir Razzaque AI/ML Engineer Portfolio

Tanzir Razzaque is an AI/ML Engineer with a strong passion for creating intelligent systems that address real-world problems. With over two years of experience in data analysis and AI research, he specializes in developing production-ready machine learning systems.

### About
- **Background**: Committed to advancing the field of artificial intelligence through practical implementation and research, focusing on deep learning and AI.
- **Interests**: Computer Vision, Large Language Models, and Agentic AI.

### Skills
- **Machine Learning & Deep Learning**: Comprehensive skills across the ML pipeline, including reinforcement learning.
- **Programming Languages**: Proficient in Python, JavaScript, C++, and Java.
- **Full Stack Development**: Expertise in React, NodeJS, and ExpressJS.
- **MLOps & Infrastructure**: Experience with Docker, Kubernetes, and AWS.

### Featured Projects
1. **Multilingual Retrieval Augmented Generation System**: An intelligent system that answers questions from PDF documents in both Bengali and English using an RAG pipeline.
2. **Pose Estimation of Vehicles in Bangladesh**: Enhances vehicle pose estimation with YOLOv8 for autonomous systems, optimized for the unique traffic conditions of Bangladesh.
3. **Heart Disease Classification**: A machine learning project predicting heart disease using multiple classification algorithms.

### Experience
- **Data Analyst** at Crown International Corporation: Analyzing data and extracting insights using SQL and data visualization tools.
- **AI/ML Researcher** at North South University: Leading the development of a vehicle pose estimation system, presenting findings at an international conference, and publishing research on IEEE Xplore.
- **FrontEnd Developer** at Crown International Corporation: Developed a high-performance responsive website and managed IT infrastructure.

### Contact
Tanzir is open to opportunities and collaborations in AI/ML. He can be reached at tanzir.razzaque@gmail.com, and is based in Dhaka, Bangladesh.

**Links**: [GitHub](#) | [LinkedIn](#)